An example that computes the representation, as in step 2 of the xNetMF algorithm given on pages 4 and 5 of the paper.

To run this notebook, ensure that config.py and xnetmf.py are in the same folder as this file. We probably shouldn't put them on GitHub, since we don't want to use any of their code.

In [1]:
import numpy as np
from scipy import sparse

from config import * # Defines Graph and RepMethod classes that we use in step 1 placeholder
from xnetmf import get_features # Computes the feature matrix in the step 1 placeholder

from representation import *

# Step 1: Node identity extraction (placeholder)

We compute the feature matrix using the paper's authors' source code.

Run only this block for two $1000 \times 1000$ adjacency matrices. Run only the next block for a specific small example found in figure 2 of the paper. Run only the third block for a sanity check on two isomorphic graphs.

In [2]:
"""np.random.seed(1)

A = sparse.csr_matrix( np.random.randint(2,size=(1000,1000)) )
B = sparse.csr_matrix( np.random.randint(2,size=(1000,1000)) )
comb = sparse.block_diag([A, B])

graph = Graph(adj = comb.tocsr())
rep_method = RepMethod(max_layer = 2)"""

'np.random.seed(1)\n\nA = sparse.csr_matrix( np.random.randint(2,size=(1000,1000)) )\nB = sparse.csr_matrix( np.random.randint(2,size=(1000,1000)) )\ncomb = sparse.block_diag([A, B])\n\ngraph = Graph(adj = comb.tocsr())\nrep_method = RepMethod(max_layer = 2)'

In [3]:
"""np.random.seed(1)

A = sparse.csr_matrix(np.array([[0., 1., 1., 1., 0.],
                                [1., 0., 0., 0., 0.],
                                [1., 0., 0., 0., 0.],
                                [1., 0., 0., 0., 1.],
                                [0., 0., 0., 1., 0.]]))

B = sparse.csr_matrix(np.array([[0., 1., 0., 0., 0., 0.],
                                [1., 0., 0., 1., 0., 0.],
                                [0., 0., 0., 1., 1., 0.],
                                [0., 1., 1., 0., 1., 1.],
                                [0., 0., 1., 1., 0., 0.],
                                [0., 0., 0., 1., 0., 0.]]))
comb = sparse.block_diag([A, B])

graph = Graph(adj = comb.tocsr())
rep_method = RepMethod(max_layer=2)"""

'np.random.seed(1)\n\nA = sparse.csr_matrix(np.array([[0., 1., 1., 1., 0.],\n                                [1., 0., 0., 0., 0.],\n                                [1., 0., 0., 0., 0.],\n                                [1., 0., 0., 0., 1.],\n                                [0., 0., 0., 1., 0.]]))\n\nB = sparse.csr_matrix(np.array([[0., 1., 0., 0., 0., 0.],\n                                [1., 0., 0., 1., 0., 0.],\n                                [0., 0., 0., 1., 1., 0.],\n                                [0., 1., 1., 0., 1., 1.],\n                                [0., 0., 1., 1., 0., 0.],\n                                [0., 0., 0., 1., 0., 0.]]))\ncomb = sparse.block_diag([A, B])\n\ngraph = Graph(adj = comb.tocsr())\nrep_method = RepMethod(max_layer=2)'

In [ ]:
np.random.seed(1)

# Same graph on 5 nodes as in the paper example.
A = sparse.csr_matrix(np.array([[0., 1., 1., 1., 0.],
                                [1., 0., 0., 0., 0.],
                                [1., 0., 0., 0., 0.],
                                [1., 0., 0., 0., 1.],
                                [0., 0., 0., 1., 0.]]))

# Permutation matrix. We can multiply perm*A*perm^T to get a new matrix whose nodes are permuted.
perm = np.array([[0., 0., 0., 0., 1.],
                 [0., 0., 0., 1., 0.],
                 [0., 0., 1., 0., 0.],
                 [0., 1., 0., 0., 0.],
                 [1., 0., 0., 0., 0.]])
# Isomorphism of first graph. Node correspondences are 0 -> 4, 1 -> 3, 2 -> 2, 3 -> 1, 4 -> 0. 
# Note that structurally, nodes 1 and 2 in the first graph and nodes 2 and 3 in the second graph are each identical. This shows up in the 
# result at the end of the notebook, where similarities are all 1.0 for each of them.
B = sparse.csr_matrix(perm @ A @ perm.T)
comb = sparse.block_diag([A, B])

graph = Graph(adj = comb.tocsr())
rep_method = RepMethod(max_layer=2)

Get the feature matrix, as computed in their source code.

In [5]:
feature_matrix = get_features(graph, rep_method, True)

max degree:  3
got k hop neighbors in time:  0.0017342567443847656
got degree sequences in time:  0.00011205673217773438


In [6]:
print(feature_matrix)

[[0.   0.21 0.1  1.  ]
 [0.   1.01 0.01 0.1 ]
 [0.   1.01 0.01 0.1 ]
 [0.   0.12 1.   0.1 ]
 [0.   1.   0.1  0.01]
 [0.   1.   0.1  0.01]
 [0.   0.12 1.   0.1 ]
 [0.   1.01 0.01 0.1 ]
 [0.   1.01 0.01 0.1 ]
 [0.   0.21 0.1  1.  ]]


# Step 2: Efficient similarity-based representation

See representation.py for the custom code (based on the source code).

First, compute the number of landmark nodes. Recall: this is, by default, the minimum of $p$ and $10\log_{2}(n)$, where $n$ is the total number of nodes of the two graphs.

In [7]:
print('Number of landmark nodes:', rep_method.p)

Number of landmark nodes: None


In [8]:
get_number_of_landmarks(graph, rep_method)
print('Number of landmark nodes:', rep_method.p)

Number of landmark nodes: 10


Get the landmark nodes, which are chosen randomly.

In [9]:
landmarks = get_random_landmarks(graph, rep_method)
print(landmarks)

[2 9 6 4 0 3 1 7 8 5]


Compute the similarity matrix between all $n$ nodes and all $p$ landmark nodes.

In [10]:
C = compute_C_matrix(feature_matrix, landmarks)
print(f'Shape of C: {len(C)} x {len(C[0])}')
print('C:')
print(C)

Shape of C: 10 x 10
C:
[[0.23267794 1.         0.19630219 0.20105033 1.         0.19630219
  0.23267794 0.23267794 0.23267794 0.20105033]
 [1.         0.23267794 0.16995867 0.98383213 0.23267794 0.16995867
  1.         1.         1.         0.98383213]
 [1.         0.23267794 0.16995867 0.98383213 0.23267794 0.16995867
  1.         1.         1.         0.98383213]
 [0.16995867 0.19630219 1.         0.20341643 0.19630219 1.
  0.16995867 0.16995867 0.16995867 0.20341643]
 [0.98383213 0.20105033 0.20341643 1.         0.20105033 0.20341643
  0.98383213 0.98383213 0.98383213 1.        ]
 [0.98383213 0.20105033 0.20341643 1.         0.20105033 0.20341643
  0.98383213 0.98383213 0.98383213 1.        ]
 [0.16995867 0.19630219 1.         0.20341643 0.19630219 1.
  0.16995867 0.16995867 0.16995867 0.20341643]
 [1.         0.23267794 0.16995867 0.98383213 0.23267794 0.16995867
  1.         1.         1.         0.98383213]
 [1.         0.23267794 0.16995867 0.98383213 0.23267794 0.16995867
  1. 

Lastly, compute representations.

In [11]:
representations_1, representations_2 = compute_representation(C, landmarks, A.shape[0])
print('Representations of nodes from first graph:')
print(representations_1)

Representations of nodes from first graph:
[[ 4.56900521e-03 -6.58431416e-01  6.72364922e-01  3.38190192e-01
  -1.71596070e-22 -4.73234254e-23 -1.03594553e-23 -9.63980388e-25
   5.59299141e-34  3.23043692e-42]
 [-5.71310364e-02 -5.55393667e-04 -1.50521002e-01  9.86954489e-01
  -5.41432549e-22 -1.79830815e-22 -1.52003362e-23 -1.26458943e-26
  -1.26549410e-33 -4.48315658e-42]
 [-5.71310364e-02 -5.55393667e-04 -1.50521002e-01  9.86954489e-01
  -5.41432549e-22 -1.79830815e-22 -1.52003362e-23 -1.26458943e-26
  -1.26549410e-33 -4.48315658e-42]
 [-4.91875859e-03  6.06340953e-01  7.42187944e-01  2.85453168e-01
  -1.36061535e-22 -4.56868336e-23 -5.48550594e-24  6.91417463e-25
  -5.98226566e-34  3.68665225e-42]
 [ 1.14762456e-01  5.16489551e-02 -1.44822949e-01  9.81421560e-01
  -5.42659225e-22 -1.79555798e-22 -1.50436178e-23 -1.30315414e-25
  -1.25827277e-33  1.99914958e-42]]


# Step 3: Fast node representation alignment

Now, we can plug this into step 3, as found in aligning.py and Example Alignments.ipynb.

In [12]:
from aligning import *

In [13]:
similarity_matrix = get_similarity_matrix(representations_1, representations_2, 4)
print(similarity_matrix)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 20 stored elements and shape (5, 5)>
  Coords	Values
  (0, 4)	1.0
  (0, 2)	0.2897295037335605
  (0, 3)	0.2897295037335605
  (0, 0)	0.2824989537646846
  (1, 2)	1.0
  (1, 3)	1.0
  (1, 0)	0.8354193765345959
  (1, 4)	0.2897295037335605
  (2, 2)	1.0
  (2, 3)	1.0
  (2, 0)	0.8354193765345959
  (2, 4)	0.2897295037335605
  (3, 1)	1.0
  (3, 0)	0.28302862283104374
  (3, 4)	0.2814413880972581
  (3, 2)	0.27570000246613857
  (4, 0)	1.0
  (4, 2)	0.8354193765345959
  (4, 3)	0.8354193765345959
  (4, 1)	0.28302862283104374
